In [118]:
# Imports

import tensorflow as tf
from tensorflow import keras, GradientTape
from keras import layers, Sequential
from keras.datasets import mnist
from keras.layers import Dense
from tensorflow import cast, float32


In [121]:
from keras import Model

model = keras.Sequential(
    [
        layers.Input(shape = (28, 28, 1)),
        layers.Conv2D(64, 3, padding='same'),
        layers.ReLU(),
        layers.Conv2D(128, 3, padding='same'),
        layers.ReLU(),
        layers.Flatten(),
        layers.Dense(10, activation='softmax')
    ],
    name = 'model'
)

cnn = Sequential([
        layers.Input(shape = (28, 28, 1)),
        layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.GlobalAveragePooling2D()
])
print(cnn.summary())

mlp = Dense(10, activation='softmax', name='classifier')
cnn.add(mlp)
print(cnn.summary())

Model: "sequential_32"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_78 (Conv2D)              │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_79 (Conv2D)              │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_32     │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,816 (73.50 KB)

 Trainable params: 18,816 (73.50 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_78 (Conv2D)              │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_79 (Conv2D)              │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_32     │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classifier (Dense)              │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,466 (76.04 KB)

 Trainable params: 19,466 (76.04 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

model = keras.Sequential(
    [
        layers.Input(shape = (28, 28, 1)),
        layers.Conv2D(64, 3, padding='same'),
        layers.ReLU(),
        layers.Conv2D(128, 3, padding='same'),
        layers.ReLU(),
        layers.Flatten(),
        layers.Dense(10, activation='softmax')
    ],
    name = 'model'
)
class CustomFit(keras.Model):
    def __init__(self, model):
        super(CustomFit, self).__init__()
        self.model = model

    def compile(self, optimizer, loss):
        super(CustomFit, self).compile()
        self.optimizer = optimizer
        self.loss = loss

    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self.model(x, training=True)
            loss = self.loss(y, y_pred)
        training_vars = self.trainable_variables
        gradients = tape.gradient(loss, training_vars)

        self.optimizer.apply_gradients(zip(gradients, training_vars))
        acc_metric.update_state(y, y_pred)

        return {"loss" : loss, "accuracy": acc_metric.result()}
    
    def test_step(self, data):
        x, y = data

        y_pred = self.model(x, training=False)
        loss = self.loss(y, y_pred)
        acc_metric.update_state(y, y_pred)

        return {"loss" : loss, "accuracy": acc_metric.result()}


acc_metric = keras.metrics.SparseCategoricalAccuracy(name="accuracy")

training = CustomFit(model)
training.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)
training.fit(x_train, y_train, batch_size=32, epochs=2)
training.evaluate(x_test, y_test, batch_size=32)

Epoch 1/2


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/nn.py:717: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 93s 49ms/step - accuracy: 0.9665 - loss: 0.0363
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 93s 50ms/step - accuracy: 0.9771 - loss: 0.0172
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9778 - loss: 6.8026e-04


[<tf.Tensor: shape=(), dtype=float32, numpy=0.0006802589632570744>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.9777846336364746>]

: 

: 

In [ ]:
def create_dataset(features, labels, batch_size):
    return tf.data.Dataset.from_tensor_slices((features, labels)).batch(batch_size)

In [ ]:
import numpy as np
from tensorflow import losses

# Mutacion del indivudo seleccionado desde el crossover
def mutate(individual, mutation_prob):
    mutated = individual.copy()

    for gene_idx in range(len(mutated)):
        if np.random.rand() < mutation_prob:
            mutated[gene_idx] = 1 - mutated[gene_idx]

    return mutated

def crossover(parent_a, parent_b):
    gene_mask = np.random.rand(len(parent_a)) < 0.5 
    return np.where(gene_mask, parent_a, parent_b)

# Seleccion de un solo individuo por torneo
def tournament_selecction(population, fitness_scores, tournament_size = 3):
    candidate_indices = np.random.choice(
        len(population),
        size = tournament_size, 
        replace = False
    )

    candidate_fitness = fitness_scores[candidate_indices]
    winner_index = candidate_indices[np.argmin(candidate_fitness)]
    return population[winner_index]

# No se si mi funcion fitness esta bien, pero es lo que saque con lo que pude
# Categorical cross-entropy la agarre porque de acuerdo a google se suele usar cuando se tienen mas de dos clases y tambien porque funciona con softmax
loss_fn = losses.CategoricalCrossentropy()
def fitness(individual, x, y, alpha, beta, classifier_model):
    if individual.sum() == 0:
        return np.inf
    individual_tf = cast(individual, float32)
    
    x_sel = x * individual_tf

    y_pred = classifier_model(x_sel, training=False)

    error = loss_fn(y, y_pred).numpy()
        
    return (
        alpha * error
        + beta * (individual.sum() / x.shape[1])
    )


def genetic_algorithm(x, y, classifier_model, pop_size=40, generations=10,
                      crossover_prob = 0.9,
                      mutation_prob = 0.2,
                      alpha=0.1, 
                      beta=0.05):
    best_individual = None
    best_fitness = np.inf
    best_fitness_history = []
    fitness_scores_history = []
    
    num_features = x.shape[1]
    # Inicializamos la poblacion 
    population = np.random.randint(0, 2, size=(pop_size, num_features))
    for generation_idx in range(generations):
        # Fitness Evaluation para toda la poblacion
        fitness_scores = np.array([
            fitness(individual, x, y, alpha, beta, classifier_model) for individual in population 
        ])
        # Seleccionamos el mejor individuo
        best_index_in_population = np.argmin(fitness_scores)
        
        # Si el fitness del mejor individuo es mejor que el best_fitness actual
        if fitness_scores[best_index_in_population] < best_fitness:
        # Actualizamos el mejor fitness y el mejor individuo
            best_fitness = fitness_scores[best_index_in_population]
            best_individual = population[best_index_in_population].copy()
        
        new_population = []

        # Seleccionamos los parents
        if np.random.rand() < 0.5:
            new_population.append(best_individual.copy())
        
        while len(new_population) < pop_size:
            parent_a = tournament_selecction(population, fitness_scores)
            parent_b = tournament_selecction(population, fitness_scores)

            if np.random.rand() < crossover_prob:
                individual = crossover(parent_a, parent_b)
            else:
                individual = parent_a.copy()
            
            individual = mutate(individual, mutation_prob)

            new_population.append(individual)
        population = np.array(new_population)

        print(f"Gen {generation_idx:02d}")
        print(f"Fitness {best_fitness:.4f}")
        print(f"Features  {best_individual.sum()}")
        print(f"Mean cost: {fitness_scores.mean():.4f}")
        print(f"Std cost: {fitness_scores.std():.6f}")
        best_fitness_history.append(best_fitness)
        fitness_scores_history.append(fitness_scores)
    return best_individual, best_fitness

# Simple GA for Dense weights
def ga_update(features, labels, dense_layer, pop_size=8, generations=10):
    W, b = dense_layer.get_weights()
    pop = [(W + np.random.randn(*W.shape)*0.1,
            b + np.random.randn(*b.shape)*0.1)
           for _ in range(pop_size)]

    best_loss = 1e9
    best = (W, b)

    for Wi, bi in pop:
        dense_layer.set_weights([Wi, bi])
        preds = dense_layer(features, training=False)
        l = loss_fn(labels, preds).numpy()
        if l < best_loss:
            best_loss = l
            best = (Wi, bi)
    dense_layer.set_weights(best)



In [115]:
def train_model(epochs, dataset, loss_fn, optimizer, feature_extractor, classifier):
    for epoch in range(epochs):
        for x, y in dataset:
            with GradientTape() as tape:
                features = feature_extractor(x, training=True)
                preds = classifier(features, training=False)
                loss = loss_fn(y, preds)
            training_vars = feature_extractor.trainable_variables
            gradients = tape.gradient(loss, training_vars)
            optimizer.apply_gradients(zip(gradients, training_vars))
            w = classifier.get_weights()[0]
            print(w.shape)
            ga_update(features.numpy(), y.numpy(), classifier)
        print(f"Epoch{epoch}")

In [116]:
# Data
(imgs_train, labels_train), (imgs_test, labels_test) = mnist.load_data()

imgs_train = imgs_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
imgs_test = imgs_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

# Modelos
cnn = Sequential([
        layers.Input(shape = (28, 28, 1)),
        layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.GlobalAveragePooling2D()
])

mlp = Dense(10, activation='softmax', name='classifier')

loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Adam()

batch_size = 5000
epochs = 1
dataset = create_dataset(imgs_train, labels_train, batch_size)
print(dataset)


<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>


In [117]:
train_model(epochs, dataset, loss_fn, optimizer, cnn, mlp)


(64, 10)
(64, 10)
(64, 10)
(64, 10)
(64, 10)
(64, 10)
(64, 10)
(64, 10)
(64, 10)
(64, 10)
(64, 10)
(64, 10)
Epoch0


2026-01-26 05:46:57.864809: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [98]:
mlp.get_weights()[0].shape

(10,)